In [77]:
import numpy as np
import pandas as pd

In [78]:
# load the titles
title = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=True).set_index('tconst')[['titleType','primaryTitle','startYear']]

C:\Users\Varun\AppData\Local\Temp\ipykernel_2292\4140932607.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=True).set_index('tconst')[['titleType','primaryTitle','startYear']]


In [79]:
title.head()

,titleType,primaryTitle,startYear
tconst,,,
tt0000001,short,Carmencita,1894
tt0000002,short,Le clown et ses chiens,1892
tt0000003,short,Pauvre Pierrot,1892
tt0000004,short,Un bon bock,1892
tt0000005,short,Blacksmith Scene,1893


In [4]:
title.dtypes

titleType       object
primaryTitle    object
startYear       object
dtype: object

In [80]:
title = title[title['startYear'] > '2000'] # filter by year

In [81]:
title.head()

,titleType,primaryTitle,startYear
tconst,,,
tt0011801,movie,Tötet nicht mehr,2019
tt0013274,movie,Istoriya grazhdanskoy voyny,2021
tt0021123,movie,The March of Time,\N
tt0034413,short,Youth Gets a Break,2001
tt0035235,short,Radio Dynamics,2016


In [82]:
# title1 = pd.read_csv('title.akas.tsv.gz', sep='\t', usecols=['titleId', 'ordering','region', 'language'], dtype={'tconst':'str', 'ordering':'int', 'region': 'str', 'language': 'str'}, na_values='\\N').set_index('titleId')
# title1 = title1[title1['region']=='IN']
# title1 = title1[title1['language']=='hi' ]

In [88]:
# load the cast of each film
cast  = pd.read_csv('title.principals.tsv.gz', sep = '\t')[['tconst', 'nconst', 'category']]
# only consider actors, not directors, composors etc.
cast = cast[cast.category.isin({'actor','actress'})]
cast.head()

,tconst,nconst,category
11,tt0000005,nm0443482,actor
12,tt0000005,nm0653042,actor
16,tt0000007,nm0179163,actor
17,tt0000007,nm0183947,actor
21,tt0000008,nm0653028,actor


In [89]:
# only consider movies, not series, etc
movies = title[title['titleType'] == 'movie']
cast = cast[cast['tconst'].isin(movies.index)]
cast.head()

,tconst,nconst,category
80702,tt0011801,nm0459029,actor
80703,tt0011801,nm0681726,actor
80704,tt0011801,nm0692612,actress
80705,tt0011801,nm0726256,actor
80706,tt0011801,nm0776458,actor


In [105]:
region = pd.read_csv('title.akas.tsv.gz',sep='\t',usecols=['titleId','region'],low_memory = True,dtype={'region':'str','titleId':'str'}).set_index('titleId')['region']
region.value_counts().head(10)

# region = pd.read_csv('title.akas.tsv.gz', sep='\t', usecols=['titleId', 'ordering','region', 'language'], dtype={'tconst':'str', 'ordering':'int', 'region': 'str', 'language': 'str'}, na_values='\\N').set_index('titleId')
# region = region[region['region']=='IN']
# region = region[region['language']=='hi' ]

DE    4225094
FR    4221318
JP    4220708
IN    4160337
ES    4142643
IT    4122839
PT    4054663
\N    1884488
US    1423787
GB     444321
Name: region, dtype: int64

In [67]:
region.head()

,ordering,region,language
titleId,,,
tt0000302,3,IN,hi
tt0011965,1,IN,hi
tt0012349,17,IN,hi
tt0013568,2,IN,hi
tt0014142,30,IN,hi


In [90]:
region_new = pd.read_csv('title.akas.tsv.gz',sep='\t',low_memory = True,dtype={'region':'str','titleId':'str'}).set_index('titleId')

C:\Users\Varun\AppData\Local\Temp\ipykernel_2292\3452743770.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  region_new = pd.read_csv('title.akas.tsv.gz',sep='\t',low_memory = True,dtype={'region':'str','titleId':'str'}).set_index('titleId')


In [91]:
region_new = region_new.loc[region_new['language']=='hi']

In [99]:
region_new.head()

,ordering,title,region,language,types,attributes,isOriginalTitle
titleId,,,,,,,
tt0000302,3,"गोभी की परी, या बच्चों का जन्म",IN,hi,imdbDisplay,\N,0
tt0011965,1,Dharma Vijay,IN,hi,alternative,\N,0
tt0012349,17,The Kid,IN,hi,imdbDisplay,\N,0
tt0013568,2,Sadhu Aur Shaitaan,IN,hi,\N,alternative spelling,0
tt0014142,30,The Hunchback of Notre Dame,IN,hi,imdbDisplay,\N,0


In [92]:
name = pd.read_csv('name.basics.tsv.gz', sep = '\t', low_memory=True, na_values = '\\N', dtype={'birthYear': float}).set_index('nconst')[['primaryName', 'birthYear']]
name.head()

,primaryName,birthYear
nconst,,
nm0000001,Fred Astaire,1899.0
nm0000002,Lauren Bacall,1924.0
nm0000003,Brigitte Bardot,1934.0
nm0000004,John Belushi,1949.0
nm0000005,Ingmar Bergman,1918.0


In [96]:
from scipy.sparse import csr_matrix

In [106]:
new_df=None
def get_pairs(lang=None, min_acted=25, min_pairings=1):
    '''
    Returns an adjacency matrix and actor mapping of actor pairs where:
    - Each actor has acted in at least min_acted films
    - The two actors have acted together in at least min_pairings films
    - And (optionally), belong to a region `lang` (IN, UN, etc)
    '''
    graph = cast
    if lang is not None:
        graph = graph[graph['tconst'].isin(region[region==lang].index)]
#         graph = graph[graph['tconst']]
                # graph = graph[graph['tconst'].isin(region[region == lang].index)]
            
    name_freq = graph['nconst'].value_counts()
    top_names = name_freq[name_freq >= min_acted]
    top_actors = graph[graph['nconst'].isin(top_names.index)]

    p = top_actors.copy()
    p['title'] = p['tconst'].astype('category')
    p['name'] = p['nconst'].astype('category')

    row = p['title'].cat.codes.values
    col = p['name'].cat.codes.values
    data = np.ones(len(p), dtype='int')

    matrix = csr_matrix((data, (row, col)))
    square = matrix.T * matrix
    square.setdiag(0)
    square = square.tocoo()

    pairs = pd.DataFrame({
        'row': square.row,
        'col': square.col,
        'n': square.data
    })
    new_df=graph
    
    pairs = pairs[pairs.n >= min_pairings].reset_index(drop=True)
    
    return graph, pairs, name.reindex(p['name'].cat.categories)

def lookup(pairs, cat):
    pairs = pd.concat([
        pairs,
        cat.iloc[pairs.row].reset_index(drop=True),
        cat.iloc[pairs.col].reset_index(drop=True),
    ], axis=1)
    pairs = pairs.drop(columns=['row', 'col'])
    pairs.columns = ['count', 'name1', 'year1', 'name2', 'year2']
    return pairs.sort_values('count', ascending=False)

In [107]:
graph, pairs, cat = get_pairs(lang='IN', min_acted=1, min_pairings=1) # setting lang = 'IN' will apply a filter allowing only INdian movies

In [108]:
ForKumu = lookup(pairs, cat)

In [109]:
ForKumu = ForKumu[['name1', 'name2', 'count']]
ForKumu = ForKumu.rename(columns={'name1':'From',
                                  'name2':'To',
                                  'count':'Strength'})


In [110]:
ForKumu.head()

,From,To,Strength
40292,Mohammad Ali,Brahmanandam,47
28041,Brahmanandam,Mohammad Ali,47
96613,Anil Nagrath,Amit Pachori,34
140069,Amit Pachori,Anil Nagrath,34
96622,Sapna Sappu,Amit Pachori,33


In [111]:
ForKumu.shape # if the kumu website crashes when you try to upload this massive dataset, try uploading a sample of this, like the first 10,000 rows.

(530994, 3)

In [113]:
writer = pd.ExcelWriter('converted-to-excel.xlsx')

In [115]:
ForKumu.to_excel(writer)

In [116]:
writer.save()

In [ ]:
merge(graph,title)